In [1]:
# NECON662D1 Assignment 2

In [97]:
# import modules
import numpy as np
import pandas as pd
import scipy as sp
import statsmodels.api as sm

In [24]:
# load data
in_dir = '/Users/jsmatte/github/ECON662D1/Assignment2'
in_file = str(in_dir + '/ass2_dat.txt')
df = pd.read_csv(in_file, sep = '\s+', header = None)
print(df)

              0          1          2          3
0    332.597273   91.52508  -7.879150  -22.82027
1     42.119419   58.32108 -10.377350  -46.89894
2    310.603232   71.19264   8.240685  -83.72890
3   1087.981690   85.37536  -7.939428  -98.93214
4    291.950898   84.63815  28.309510  -80.68243
..          ...        ...        ...        ...
95  1696.762100   26.54987 -40.947840 -146.96500
96   728.001441   69.94029  75.729620 -126.41690
97   539.483943   89.50351 -44.335000  -76.11952
98    21.440609  100.67600  48.674720  -16.33207
99   305.192266  133.57730   4.789128  -10.57532

[100 rows x 4 columns]


In [26]:
df.columns = ['y', 'x1', 'x2', 'x3']
print(df)

              y         x1         x2         x3
0    332.597273   91.52508  -7.879150  -22.82027
1     42.119419   58.32108 -10.377350  -46.89894
2    310.603232   71.19264   8.240685  -83.72890
3   1087.981690   85.37536  -7.939428  -98.93214
4    291.950898   84.63815  28.309510  -80.68243
..          ...        ...        ...        ...
95  1696.762100   26.54987 -40.947840 -146.96500
96   728.001441   69.94029  75.729620 -126.41690
97   539.483943   89.50351 -44.335000  -76.11952
98    21.440609  100.67600  48.674720  -16.33207
99   305.192266  133.57730   4.789128  -10.57532

[100 rows x 4 columns]


In [35]:
# create matrix X of the regressors
X = np.zeros((len(df),4))
for i in range(X.shape[0]):
    X[i] = [1, df.x1.values[i], df.x2.values[i], df.x3.values[i]]
print(X.shape)

# create vector y of observations
y = np.array([[j] for j in df.y.values])
print(y.shape)

# Regression Model
model = sm.OLS(y, X).fit()
print(model.summary())
print()
print('Residual Std. Error: ',  np.sqrt(model.scale))

(100, 4)
(100, 1)
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.626
Model:                            OLS   Adj. R-squared:                  0.614
Method:                 Least Squares   F-statistic:                     53.54
Date:                Wed, 11 Nov 2020   Prob (F-statistic):           2.00e-20
Time:                        17:08:53   Log-Likelihood:                -679.21
No. Observations:                 100   AIC:                             1366.
Df Residuals:                      96   BIC:                             1377.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -307.9884    117.223  

In [62]:
# Model parameters
print(model.params)
beta0 = model.params[0]
beta1 = model.params[1]
beta2 = model.params[2]
beta3 = model.params[3]

[-307.98841316    3.91597727   -4.02059773   -8.48159487]


In [89]:
# predictions, residuals and disturbances
pred = model.fittedvalues
print(pred.shape)

resid = model.resid
print(resid.shape)

dist = []
for m in range(len(y)):
    dist.append(y[m][0] - beta0 - beta1*X[m][1] - beta2*X[m][2] - beta3*X[m][3])
print(len(dist))

(100,)
(100,)
100


In [90]:
# a) estimates of the covariance matrix of \hat{\beta}
# i) standard OLS estimate
# ii) HCo, HC1, HC2, HC3, HC4
# iii) Newwy-West HAC

In [91]:
# i) standard OLS estimate
sum_resid_squared = 0
for r in resid:
    sum_resid_squared += r**2
Vhat_i = (1 / (len(y) - 4)) * sum_resid_squared * np.linalg.inv(np.matmul(X.T, X))
print(Vhat_i)
print('')
print(model.cov_params())

[[ 1.37412989e+04 -1.14577856e+02  8.04444087e+00  6.76760297e+01]
 [-1.14577856e+02  1.10947664e+00 -1.64873513e-01 -4.22071133e-01]
 [ 8.04444087e+00 -1.64873513e-01  8.34828446e-01 -1.68916430e-02]
 [ 6.76760297e+01 -4.22071133e-01 -1.68916430e-02  5.74773180e-01]]

[[ 1.37412989e+04 -1.14577856e+02  8.04444087e+00  6.76760297e+01]
 [-1.14577856e+02  1.10947664e+00 -1.64873513e-01 -4.22071133e-01]
 [ 8.04444087e+00 -1.64873513e-01  8.34828446e-01 -1.68916430e-02]
 [ 6.76760297e+01 -4.22071133e-01 -1.68916430e-02  5.74773180e-01]]


In [92]:
# ii) HC0, HC1, HC2, HC3, HC4
print('HC0')
Ohat_HC0 = np.zeros((len(y),len(y)))
for i0 in range(len(y)):
    Ohat_HC0[i0][i0] = resid[i0]**2
    
Vhat_ii_HC0 = np.matmul(np.matmul(np.matmul(np.matmul(np.linalg.inv(np.matmul(X.T, X)), X.T), Ohat_HC0), X), np.linalg.inv(np.matmul(X.T, X)))
print(Vhat_ii_HC0)
print('')
print(model.cov_HC0)
print('')

print('HC1')
Ohat_HC1 = np.zeros((len(y),len(y)))
for i1 in range(len(y)):
    Ohat_HC1[i1][i1] = dist[i1]**2
    
Vhat_ii_HC1 = (len(y) / (len(y) - 4)) * np.matmul(np.matmul(np.matmul(np.matmul(np.linalg.inv(np.matmul(X.T, X)), X.T), Ohat_HC1), X), np.linalg.inv(np.matmul(X.T, X)))
print(Vhat_ii_HC1)
print('')
print(model.cov_HC1)
print('')

print('HC2')
Px = np.matmul(X, np.matmul(np.linalg.inv(np.matmul(X.T, X)), X.T))

Ohat_HC2 = np.zeros((len(y),len(y)))
for i2 in range(len(y)):
    Ohat_HC2[i2][i2] = resid[i2]**2 / (1 - Px[i2][i2])
    
Vhat_ii_HC2 = np.matmul(np.matmul(np.matmul(np.matmul(np.linalg.inv(np.matmul(X.T, X)), X.T), Ohat_HC2), X), np.linalg.inv(np.matmul(X.T, X)))
print(Vhat_ii_HC2)
print('')
print(model.cov_HC2)
print('')

print('HC3')
Ohat_HC3 = np.zeros((len(y),len(y)))
for i3 in range(len(y)):
    Ohat_HC3[i3][i3] = resid[i3]**2 / ((1 - Px[i3][i3])**2)
    
Vhat_ii_HC3 = np.matmul(np.matmul(np.matmul(np.matmul(np.linalg.inv(np.matmul(X.T, X)), X.T), Ohat_HC3), X), np.linalg.inv(np.matmul(X.T, X)))
print(Vhat_ii_HC3)
print('')
print(model.cov_HC3)
print('')

HC0
[[ 1.24530797e+04 -9.57197205e+01 -3.72882783e+00  8.31958443e+01]
 [-9.57197205e+01  8.66960180e-01  6.63431413e-02 -4.32988909e-01]
 [-3.72882783e+00  6.63431413e-02  5.68223212e-01  1.48052795e-01]
 [ 8.31958443e+01 -4.32988909e-01  1.48052795e-01  1.03751374e+00]]

[[ 1.24530797e+04 -9.57197205e+01 -3.72882783e+00  8.31958443e+01]
 [-9.57197205e+01  8.66960180e-01  6.63431413e-02 -4.32988909e-01]
 [-3.72882783e+00  6.63431413e-02  5.68223212e-01  1.48052795e-01]
 [ 8.31958443e+01 -4.32988909e-01  1.48052795e-01  1.03751374e+00]]

HC1
[[ 1.29719580e+04 -9.97080422e+01 -3.88419566e+00  8.66623378e+01]
 [-9.97080422e+01  9.03083521e-01  6.91074389e-02 -4.51030114e-01]
 [-3.88419566e+00  6.91074389e-02  5.91899179e-01  1.54221662e-01]
 [ 8.66623378e+01 -4.51030114e-01  1.54221662e-01  1.08074348e+00]]

[[ 1.29719580e+04 -9.97080422e+01 -3.88419566e+00  8.66623378e+01]
 [-9.97080422e+01  9.03083521e-01  6.91074389e-02 -4.51030114e-01]
 [-3.88419566e+00  6.91074389e-02  5.91899179e-0

In [93]:
# iii) Newwy-West HAC

In [94]:
# 2 standard error of the estimate of coefficient x3 (i.e. standard error of \hat{\beta_{3}})

In [102]:
# i) standard OLS estimate
sehat_i = []
for ii00 in range(Vhat_i.shape[0]):
    sehat_i.append(np.sqrt(Vhat_i[ii00][ii00]))
print(sehat_i[3])
print(model.bse[3])

0.7581379691371817
0.7581379691371765


In [104]:
# ii) HC0, HC1, HC2, HC3, HC4
print('HC0')
sehat_ii_HC0 = []
for ii0 in range(Vhat_ii_HC0.shape[0]):
    sehat_ii_HC0.append(np.sqrt(Vhat_ii_HC0[ii0][ii0]))
print(sehat_ii_HC0[3])
print(model.HC0_se[3])
print('')

print('HC1')
sehat_ii_HC1 = []
for ii1 in range(Vhat_ii_HC1.shape[0]):
    sehat_ii_HC1.append(np.sqrt(Vhat_ii_HC1[ii1][ii1]))
print(sehat_ii_HC1[3])
print(model.HC1_se[3])
print('')

print('HC2')
sehat_ii_HC2 = []
for ii2 in range(Vhat_ii_HC2.shape[0]):
    sehat_ii_HC2.append(np.sqrt(Vhat_ii_HC2[ii2][ii2]))
print(sehat_ii_HC2[3])
print(model.HC2_se[3])
print('')

print('HC3')
sehat_ii_HC3 = []
for ii3 in range(Vhat_ii_HC3.shape[0]):
    sehat_ii_HC3.append(np.sqrt(Vhat_ii_HC3[ii3][ii3]))
print(sehat_ii_HC3[3])
print(model.HC3_se[3])
print('')

HC0
1.0185841832968705
1.0185841832968607

HC1
1.039588128811194
1.0395881288111837

HC2
1.0580544663538576
1.058054466353847

HC3
1.0993814641866588
1.0993814641866477



In [105]:
# iii) Newwy-West HAC